In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print('Ready to use {}, {}'.format(azureml.core.VERSION, ws.name))

Ready to use 1.42.0, ml-azure


In [2]:
import os, shutil

train_1 = 'test-for'

os.makedirs(train_1, exist_ok =True)
shutil.copy('data/diabetes.csv', os.path.join(train_1, 'diabetes.csv'))

'test-for/diabetes.csv'

In [26]:
%%writefile $train_1/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Overwriting test-for/diabetes_training.py


In [27]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

env = Environment.from_conda_specification('experiment', 'environment.yml')

script_config = ScriptRunConfig(source_directory = train_1,
                               script = 'diabetes_training.py',
                               environment = env,
                               docker_runtime_config = DockerConfiguration(use_docker=True))

experiment_name = 'trial_experiment'
experiment =Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'trial_experiment_1658757446_7d54d9f9',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-07-25T13:57:28.107627Z',
 'endTimeUtc': '2022-07-25T13:57:39.278746Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'e0a6f17f-ceb1-45c2-ad28-0d0cabaa5006'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'experiment',
   'version': 'Autosave_2022-07-25T13:22:51Z_821a1afa',
   'assetId': 'azureml://locations/eastus2/workspaces/36da6dff

In [28]:

metrics = run.get_metrics()

for key in metrics.keys():
    print(key, metrics.get(key))
    
for file in run.get_file_names():
    print(file)

Regularization Rate 0.01
Accuracy 0.774
AUC 0.8483441962286681
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/8_azureml.log
outputs/diabetes_model.pkl


In [31]:
from azureml.core import Model

model = run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version: ', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t', tag_name, ':', tag)
    for properties_name in model.properties:
        properties = model.properties[properties_name]
        print('\t', properties_name, ':', properties)
    print('\n')

diabetes_model version:  3
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774


diabetes_model version:  2
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774


diabetes_model version:  1
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774




In [6]:
%%writefile $train_1/diabetes_training_ar.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

argparser = argparse.ArgumentParser()
argparser.add_argument('--reg_rate', type = float, dest = 'reg', default=0.01)
args = argparser.parse_args()
reg = args.reg

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)


# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Overwriting test-for/diabetes_training_ar.py


In [7]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

env = Environment.from_conda_specification('experiment', 'environment.yml')

script_config = ScriptRunConfig(source_directory = train_1,
                               script = 'diabetes_training_ar.py',
                                arguments = ['--reg_rate', 0.1],
                               environment = env,
                               docker_runtime_config = DockerConfiguration(use_docker=True))

experiment_name = 'trial_experiment_ar'
experiment =Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'trial_experiment_ar_1658785340_7558a7ea',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-07-25T21:42:22.322945Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '270855bc-0c60-4631-a2d0-9515bb1caa06'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training_ar.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg_rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'experiment',
   'version': 'Autosave_2022-07-25T13:22:51Z_821a1afa',
   'assetId': 'azureml://locations/eastus2/workspaces/36da6dff-9597-4be8-87ee-cef9

In [8]:
metrics = run.get_metrics()

for key in metrics.keys():
    print(key, metrics.get(key))
    
for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.774
AUC 0.8484377332205582
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/8_azureml.log
outputs/diabetes_model.pkl


In [9]:
from azureml.core import Model

model = run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version: ', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t', tag_name, ':', tag)
    for properties_name in model.properties:
        properties = model.properties[properties_name]
        print('\t', properties_name, ':', properties)
    print('\n')

diabetes_model version:  4
	 Training context : Script
	 AUC : 0.8484377332205582
	 Accuracy : 0.774


diabetes_model version:  3
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774


diabetes_model version:  2
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774


diabetes_model version:  1
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774


